In [1]:
import re
import sys
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
sys.path.append('..')

In [3]:
from ingestion.ingestion_mediator import IngestionMediator
mediator = IngestionMediator()

/home/johnny/jk-apps/bike-price-predictor/env/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [7]:
specs_df = pd.read_csv('../data/raw_data/12152019/specialized_specs_all.csv')
specs_df.head()

,rims,saddle,shift_levers,bell,inner_tubes,fenders,ui_remote,chainrings,grips,site,taillight,front_tire,rear_brake,product_id,crankset,tape,front_hub,handlebars,rear_wheel,seat_binder,stem,rack,seatpost,front_wheel,charger,front_brake,rear_tire,pedals,chain,cassette,motor,spokes,kickstand,rear_derailleur,battery,fork,front_derailleur,frame,headlight,bottom_bracket,wiring_harness,rear_shock,rear_hub
0,"Roval Control SL, hookless carbon, 25mm intern...","Body Geometry S-Works Power, carbon fiber rail...","SRAM XX1 Eagle AXS, trigger, 12-speed",NaN,"Presta, 60mm valve",NaN,NaN,"Alloy, 32T",Specialized Trail Grips,specialized,NaN,"Fast Trak, Control casing, GRIPTON® compound, ...","SRAM Level Ultimate, hydraulic disc",171229,"QUARQ XX1 Eagle Power Meter, Boost™ 148, DUB, ...",NaN,"Roval Control SL, sealed cartridge bearings, 1...","S-Works Carbon XC Mini Rise, 6-degree upsweep,...",NaN,"Alloy, 34.9mm, titanium bolt","S-Works SL, alloy, titanium bolts, 6-degree rise",NaN,"S-Works FACT carbon, 10mm setback, 30.9mm",NaN,NaN,"SRAM Level Ultimate, hydraulic disc","Fast Trak, Control casing, GRIPTON® compound, ...",Specialized Dirt,SRAM XX1 (Rainbow),"SRAM XG-1299 Eagle, 10-50t",NaN,DT Swiss Competition Race,NaN,SRAM XX1 Eagle AXS,NaN,"RockShox SID Brain Ultimate, top-adjust Brain ...",NaN,"S-Works FACT 12m, XC Geometry, Rider-First Eng...",NaN,"SRAM DUB, threaded BB",NaN,Custom RockShox Micro Brain shock w/ Spike Val...,"Roval Control SL, DT Swiss Star Ratchet, 54t e..."
1,"Roval Traverse 29, hookless alloy, 30mm inner ...","Body Geometry Henge Comp, hollow Cr-Mo rails, ...","SRAM NX Eagle, trigger, 12-speed",NaN,"Standard, Presta valve",NaN,NaN,30T,"Deity, Knuckleduster, Black",specialized,NaN,"Butcher, GRID TRAIL casing, GRIPTON® compound,...","SRAM Code R, 4-piston caliper, hydraulic disc,...",173494,"SRAM NX Eagle, DUB, 165mm, 30T",NaN,"Specialized, sealed cartridge bearings, 15x110...","Specialized, 7050 alloy, 6-degree upsweep, 8-d...",NaN,"Specialized bolt-type, alloy, 38.6mm","Specialized Trail, 3D-forged alloy, 4-bolt, 6-...",NaN,"X-Fusion Manic, infinite adjustable, two-bolt ...",NaN,NaN,"SRAM Code R, 4-piston caliper, hydraulic disc,...","Butcher, GRID TRAIL casing, GRIPTON® compound,...",Specialized Dirt,"SRAM NX Eagle, 12-speed","SRAM NX Eagle, 12-speed, 11-50t",NaN,DT Swiss Industry,NaN,"SRAM NX Eagle, 12-speed",NaN,"FOX FLOAT 36 Rhythm, GRIP damper, two position...",NaN,"FACT 11m full carbon frame, asymmetrical desig...",NaN,"SRAM DUB, threaded BB",NaN,"FOX FLOAT DPX2 Performance, Rx Trail Tune, EVO...","Specialized, sealed cartridge bearings, 12x148..."
2,"Roval Traverse 27.5, hookless alloy, 30mm inne...","Body Geometry Henge Comp, hollow Cr-Mo rails, ...","SRAM NX Eagle, trigger, 12-speed",NaN,"Standard, Presta valve",NaN,NaN,32T,"Deity, Knuckleduster, Black",specialized,NaN,"Butcher, GRID casing, GRIPTON® compound, 2Blis...","SRAM Code R, 4-piston caliper, hydraulic disc,...",173495,"SRAM NX Eagle, DUB, 165mm, 32T",NaN,"Specialized, sealed cartridge bearings, 15x110...","Specialized, 7050 alloy, 6-degree upsweep, 8-d...",NaN,"Specialized bolt-type, alloy, 38.6mm","Specialized Trail, 3D-forged alloy, 4-bolt, 6-...",NaN,"X-Fusion Manic, infinite adjustable, two-bolt ...",NaN,NaN,"SRAM Code R, 4-piston caliper, hydraulic disc,...","Butcher, GRID casing, GRIPTON® compound, 2Blis...",Specialized Dirt,"SRAM NX Eagle, 12-speed","SRAM NX Eagle, 12-speed, 11-50t",NaN,DT Swiss Industry,NaN,"SRAM NX Eagle, 12-speed",NaN,"FOX FLOAT 36 Rhythm, Grip damper, 2-position s...",NaN,"FACT 11m full carbon frame, asymmetrical desig...",NaN,"SRAM DUB, threaded BB",NaN,"FOX FLOAT DPX2 Performance, Rx Trail Tune, EVO...","Specialized, sealed cartridge bearings, 12x148..."
3,"Alloy SL, disc only, double-wall, 29mm interna...","Bridge Comp, Hollow Cr-mo rails, 155/143mm","SRAM NX Eagle, trigger, 12-speed",NaN,"Standard, Presta valve",NaN,NaN,"Steel, 30T",Specialized Trail Grips,specialized,NaN,"Butcher, GRID TRAIL casing, GRIPTON® compound,..

In [8]:
sorted(specs_df.columns)

['battery',
 'bell',
 'bottom_bracket',
 'cassette',
 'chain',
 'chainrings',
 'charger',
 'crankset',
 'fenders',
 'fork',
 'frame',
 'front_brake',
 'front_derailleur',
 'front_hub',
 'front_tire',
 'front_wheel',
 'grips',
 'handlebars',
 'headlight',
 'inner_tubes',
 'kickstand',
 'motor',
 'pedals',
 'product_id',
 'rack',
 'rear_brake',
 'rear_derailleur',
 'rear_hub',
 'rear_shock',
 'rear_tire',
 'rear_wheel',
 'rims',
 'saddle',
 'seat_binder',
 'seatpost',
 'shift_levers',
 'site',
 'spokes',
 'stem',
 'taillight',
 'tape',
 'ui_remote',
 'wiring_harness']

In [11]:
specs_df[['front_wheel', 'rear_wheel']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378 entries, 0 to 377
Data columns (total 2 columns):
front_wheel    104 non-null object
rear_wheel     102 non-null object
dtypes: object(2)
memory usage: 6.0+ KB


In [12]:
specs_df.front_wheel.unique()

array([nan, 'WH-TRAVERSE SL 29 148 XX1 OHLINS', 'Roval CLX 32 Disc',
       'Roval CL50 Disc, carbon, tubeless-ready, Win Tunnel Engineered, 50mm depth, 21h',
       'Roval CL 50 Disc, carbon, tubeless-ready, Win Tunnel Engineered, 50mm depth, 21h',
       'Roval CLX 64 Disc, carbon, tubeless-ready, Win Tunnel Engineered, 64mm depth, CeramicSpeed bearings, 21h',
       'Roval CLX 50',
       'Roval C 38 Disc, carbon, tubeless-ready, 38mm depth, 24h',
       'Roval CLX 50 Disc, carbon, tubeless-ready, Win Tunnel Engineered, 50mm depth, CeramicSpeed bearings, 21h',
       'Roval CLX 64 Disc rim, Roval AFD1, Center Lock™, CeramicSpeed bearings, 21h',
       'Roval CL 32 Disc', 'Roval C 38 Disc',
       'DT R470 Disc, sealed cartridge hubs, 14g spokes, 24h',
       'Axis Sport Disc',
       'DT R460, sealed cartridge hubs, 14g spokes, 20h', 'DT R470 Disc',
       'Roval C38 Disc', 'Roval CLX 50 - Sagan LTD',
       'Roval CLX 32 Disc, Win Tunnel Engineered, carbon rim, 32mm depth,\xa0Roval

In [8]:
site = 'nashbar'
munged_df = mediator.transform_raw_data(source=site)
munged_df

[Manifest._validate_from_list()]


{'site': 'nashbar',
 'tablename': 'munged',
 'filename': 'nashbar_munged.csv',
 'timestamp': '01012020',
 'loaded': False,
 'date_loaded': None}

In [9]:
date = munged_df['timestamp']
filename = munged_df['filename']
munged_path = f'../data/munged_data/{date}/{filename}'
munged_df = pd.read_csv(munged_path)
munged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 21 columns):
site                  97 non-null object
bike_type             97 non-null object
product_id            97 non-null int64
description           97 non-null object
brand                 97 non-null object
href                  97 non-null object
price                 87 non-null float64
msrp                  97 non-null float64
model_year            36 non-null float64
frame_material        78 non-null object
handlebar_material    74 non-null object
fd_groupset           37 non-null object
rd_groupset           62 non-null object
cassette_groupset     72 non-null object
crankset_material     39 non-null object
crankset_groupset     30 non-null object
brake_type            70 non-null object
seatpost_material     77 non-null object
fork_material         60 non-null object
chain_groupset        19 non-null object
shifter_groupset      64 non-null object
dtypes: float64(3), int64(1), obje

In [ ]:
null_ids = munged_df.loc[munged_df.fork_material.isnull(), 'product_id']
tmp = specs_df.loc[specs_df.product_id.isin(null_ids), ['product_id', 'fork', 'fork_material', 'forks',
                                                       'frame_fork', 'material', 'frameset_material']]
tmp[tmp.fork.notnull()]

In [ ]:
tmp.fork.value_counts()

In [ ]:
tmp.info()

In [ ]:
prod_df = pd.read_csv('../data/raw_data/12152019/wiggle_prods_all.csv')
s_df = pd.read_csv('../data/raw_data/01012020/wiggle_specs_all.csv')

In [ ]:
merge_df = prod_df.merge(right=s_df, on='product_id')
merge_df.head()

In [4]:
mediator.aggregate_data()

In [5]:
agg_df = pd.read_csv('../data/munged_data/combined/combined_munged_01012020.csv')
agg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9596 entries, 0 to 9595
Data columns (total 22 columns):
site                  9596 non-null object
bike_type             9596 non-null object
product_id            9596 non-null object
href                  9596 non-null object
description           9596 non-null object
brand                 9596 non-null object
price                 9194 non-null float64
msrp                  9596 non-null float64
frame_material        8566 non-null object
model_year            3831 non-null float64
brake_type            8363 non-null object
fork_material         4617 non-null object
handlebar_material    4734 non-null object
fd_groupset           3356 non-null object
rd_groupset           7483 non-null object
cassette_groupset     7183 non-null object
crankset_material     2493 non-null object
crankset_groupset     5008 non-null object
seatpost_material     4264 non-null object
chain_groupset        4712 non-null object
shifter_groupset      6913 non

In [32]:
agg_df.loc[agg_df.price_x.notnull(), ['site', 'price', 'price_x']].site.unique()

array(['bicycle_warehouse'], dtype=object)

In [19]:
import os
prods = [p for p in os.listdir('../data/raw_data/12152019') if 'prods' in p]
prods

['proshop_prods_all.csv',
 'spokes_prods_all.csv',
 'rei_prods_all.csv',
 'litespeed_prods_all.csv',
 'competitive_prods_all.csv',
 'canyon_prods_all.csv',
 'contebikes_prods_all.csv',
 'jenson_prods_all.csv',
 'eriks_prods_all.csv',
 'nashbar_prods_all.csv',
 'trek_prods_all.csv',
 'bicycle_warehouse_prods_all.csv',
 'citybikes_prods_all.csv',
 'lynskey_prods_all.csv',
 'wiggle_prods_all.csv',
 'backcountry_prods_all.csv',
 'specialized_prods_all.csv']

In [20]:
csv = prods.pop()
f_path = f'../data/raw_data/12152019/{csv}'
prods_agg = pd.read_csv(f_path)

In [21]:
prods_agg.head()

,site,bike_type,brand,product_id,href,description,price,msrp
0,specialized,mountain,specialized,171229,/us/en/s-works-epic-axs/p/171229?color=264122-...,S-Works Epic AXS,11020.0,11020.0
1,specialized,mountain,specialized,173494,/us/en/stumpjumper-evo-comp-carbon-29/p/173494...,Stumpjumper EVO Comp Carbon 29,4520.0,4520.0
2,specialized,mountain,specialized,173495,/us/en/stumpjumper-evo-comp-carbon-27-5/p/1734...,Stumpjumper EVO Comp Carbon 27.5,4520.0,4520.0
3,specialized,mountain,specialized,171068,/us/en/fuse-expert-29/p/171068?color=263307-17...,Fuse Expert 29,2150.0,2150.0
4,specialized,mountain,specialized,171069,/us/en/fuse-comp-29/p/171069?color=263340-171069,Fuse Comp 29,1675.0,1675.0


In [25]:
for f in prods:
    f_path = f'../data/raw_data/12152019/{f}'
    tmp_df = pd.read_csv(f_path)
    prods_agg = prods_agg.append(tmp_df, ignore_index=True, sort=False)
    
prods_agg.head()

,site,bike_type,brand,product_id,href,description,price,msrp
0,specialized,mountain,specialized,171229,/us/en/s-works-epic-axs/p/171229?color=264122-...,S-Works Epic AXS,11020.0,11020.0
1,specialized,mountain,specialized,173494,/us/en/stumpjumper-evo-comp-carbon-29/p/173494...,Stumpjumper EVO Comp Carbon 29,4520.0,4520.0
2,specialized,mountain,specialized,173495,/us/en/stumpjumper-evo-comp-carbon-27-5/p/1734...,Stumpjumper EVO Comp Carbon 27.5,4520.0,4520.0
3,specialized,mountain,specialized,171068,/us/en/fuse-expert-29/p/171068?color=263307-17...,Fuse Expert 29,2150.0,2150.0
4,specialized,mountain,specialized,171069,/us/en/fuse-comp-29/p/171069?color=263340-171069,Fuse Comp 29,1675.0,1675.0


In [26]:
prods_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8051 entries, 0 to 8050
Data columns (total 8 columns):
site           8051 non-null object
bike_type      8051 non-null object
brand          8051 non-null object
product_id     8051 non-null object
href           8051 non-null object
description    8051 non-null object
price          8013 non-null float64
msrp           8051 non-null float64
dtypes: float64(2), object(6)
memory usage: 503.3+ KB


In [27]:
prods_agg.loc[prods_agg.price.isnull()]

,site,bike_type,brand,product_id,href,description,price,msrp
5120,eriks,road_bikes,Specialized,PR3E16197,https://www.eriksbikeshop.com/specialized-2019...,Specialized 2019 Diverge E5 Elite Women's Road...,NaN,1399.99
5122,eriks,road_bikes,Specialized,PR3E16200,https://www.eriksbikeshop.com/specialized-2019...,Specialized 2019 Diverge E5 Base Road Bike,NaN,1099.99
5123,eriks,road_bikes,Specialized,PR3E16190,https://www.eriksbikeshop.com/specialized-2019...,Specialized 2019 Tarmac Expert Disc Road Bike,NaN,4749.99
5124,eriks,road_bikes,Specialized,PR3E16191,https://www.eriksbikeshop.com/specialized-2019...,Specialized 2019 Tarmac Comp Road Bike,NaN,3499.99
5126,eriks,road_bikes,Specialized,PR3E16179,https://www.eriksbikeshop.com/specialized-2019...,Specialized 2019 Diverge E5 Comp Road Bike,NaN,1899.99
5127,eriks,road_bikes,Specialized,PR3E16180,https://www.eriksbikeshop.com/specialized-2019...,Specialized 2019 Diverge Carbon Road Bike,NaN,2499.99
5156,eriks,road_bikes,Specialized,PR3E13242,https://www.eriksbikeshop.com/specialized-2019...,Specialized 2019 Ruby Base Women's Road Bike,NaN,1899.99
5158,eriks,road_bikes,Specialized,PR3E12725,https://www.eriksbikeshop.com/specialized-2019...,Specialized 2019 Allez Base Road Bike,NaN,839.99
5168,eriks,road_bikes,Specialized,PR3E12729,https://www.eriksbikeshop.com/specialized-2018...,Specialized 2018 Dolce Base Women's Road Bike,NaN,799.99
5390,eriks,mountain_bikes,Specialized,PR3E16645,https://www.eriksbikeshop.com/specialized-2019...,Specialized 2019 Women's Rockhopper Sport 29er...,NaN,709.99
